In [2]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob

In [3]:
fs = fsspec.filesystem('s3', anon=True)

In [23]:
urls = ['s3://' + f for f in fs.glob("s3://noaa-goes16/ABI-L1b-RadC/2000/001/*/*.nc")]

In [24]:
import dask

In [25]:
from dask.distributed import Client

In [26]:
client = Client(n_workers=30)

/home/vboxuser/anaconda3/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33097 instead
  warnings.warn(


In [27]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33097/status,
Dashboard: http://127.0.0.1:33097/status,Workers: 30
Total threads: 30,Total memory: 15.32 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42891,Workers: 30
Dashboard: http://127.0.0.1:33097/status,Total threads: 30
Started: Just now,Total memory: 15.32 GiB
Comm: tcp://127.0.0.1:43191,Total threads: 1
Dashboard: http://127.0.0.1:40849/status,Memory: 522.93 MiB
Nanny: tcp://127.0.0.1:42899,


In [28]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
           outf.write(ujson.dumps(h5chunks.translate()).encode())

In [29]:
import pathlib

In [30]:
pathlib.Path('./jsons/').mkdir(exist_ok=True)

In [31]:
dask.compute(*[dask.delayed(gen_json)(u) for u in urls])

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [33]:
json_list = sorted(glob('./jsons/*.json'))

In [34]:
mzz = MultiZarrToZarr(
    json_list,
    remote_protocol="s3",
    remote_options={'anon':True},
    concat_dims='t',
    inline_threshold=0
)

In [35]:
mzz.translate('./combined.json')

/home/vboxuser/anaconda3/lib/python3.11/site-packages/kerchunk/combine.py:370: UserWarning: Concatenated coordinate 't' contains less than expectednumber of values across the datasets: [-999.]
  warnings.warn(


{'version': 1,
 'refs': {'.zgroup': '{"zarr_format":2}',
  't/.zarray': '{\n    "chunks": [\n        1\n    ],\n    "compressor": null,\n    "dtype": "<f8",\n    "fill_value": null,\n    "filters": null,\n    "order": "C",\n    "shape": [\n        1\n    ],\n    "zarr_format": 2\n}',
  't/0': 'base64:AAAAAAA4j8A=',
  't/.zattrs': '{\n    "_ARRAY_DIMENSIONS": [\n        "t"\n    ],\n    "axis": "T",\n    "bounds": "time_bounds",\n    "long_name": "J2000 epoch mid-point between the start and end image scan in seconds",\n    "standard_name": "time",\n    "units": "seconds since 2000-01-01 12:00:00"\n}',
  '.zattrs': '{"Conventions":"CF-1.7","Metadata_Conventions":"Unidata Dataset Discovery v1.0","cdm_data_type":"Image","dataset_name":"OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170671748180.nc","date_created":"2017-03-08T17:48:18.0Z","id":"0cf12833-7989-4c85-97fa-dbc022b9f3de","institution":"DOC\\/NOAA\\/NESDIS > U.S. Department of Commerce, National Oceanic and Atmospher

In [36]:
fs = fsspec.filesystem(
    "reference", 
    fo="./combined.json", 
    remote_protocol="s3", 
    remote_options={"anon":True},
    skip_instance_cache=True
)

In [37]:
m = fs.get_mapper("")

In [38]:
ds = xr.open_dataset(m, engine='zarr')

/tmp/ipykernel_10519/2381503581.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_dataset(m, engine='zarr')
